## 背景
随着网络诈骗越来越多，经常有一些不法分子利用网络会议软件进行诈骗，为此需要训练一个文本分类检测模型，来实时检测会议中的对话内容是否存在诈骗风险，以帮助用户在网络会议中增强警惕，减少受到欺诈的风险。

考虑模型的预训练成本高昂，选择一个通用能力强的模型底座进行微调是一个比较经济的做法，我们选择模型底座主要考虑以下几个因素：
- 预训练主要采用中文语料，具有良好的中文支持能力
- 模型需要具备基本的指令遵循能力。
- 模型要能理解json格式，具备输出json格式的能力。
- 在满足以上几个能力的基础上，模型参数量越小越好。

选厂商：
- 智谱AI：清华，参数规模主要在6B、9B
- 书生浦语：商汤，只有一个7B规模的参数
- 零一万物：创新工厂，只有一个6B模型
- 通义千问：阿里，参数规模有0.5B、1.5B、7B、72B

通义千问Qwen2提供了不同参数大小的模型，能让我们有更多的选择。我们需要做的是从大到小依次测试每个模型的能力，找到满足自己需要的最小参数模型。

In [5]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## 模型下载

依次下载qwen2的不同尺寸的模型：0.5B-Instruct、1.5B、1.5B-Instruct、7B-Instruct，下载完后会输出模型在本地磁盘的路径。
> 1.5B和1.5B-Instruct的主要区别：按照官方文档，前者是预训练模型，后者是经过指令微调的模型。为了确定应该选择预训练模型还是指令微调模型，这里将两个模型都下载下来进行对比测试。

In [7]:
#模型下载
from modelscope import snapshot_download
cache_dir = '/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/'
# model_dir = snapshot_download('Qwen/Qwen2-7B-Instruct', cache_dir=cache_dir, revision='master')
# model_dir = snapshot_download('Qwen/Qwen2-0.5B-Instruct', cache_dir=cache_dir, revision='master')
# model_dir = snapshot_download('Qwen/Qwen2-1.5B-Instruct', cache_dir=cache_dir, revision='master')
model_dir = snapshot_download('Qwen/Qwen2-1.5B', cache_dir=cache_dir, revision='master')
model_dir

2025-03-02 02:35:29,020 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-03-02 02:35:29,262 - modelscope - INFO - Got 10 files, start to download ...


Processing 10 items:   0%|          | 0.00/10.0 [00:00<?, ?it/s]

2025-03-02 02:45:12,118 - modelscope - INFO - Download model 'Qwen/Qwen2-1.5B' successfully.
2025-03-02 02:45:12,119 - modelscope - INFO - Creating symbolic link [/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1.5B].


'/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B'

In [1]:
!ls -l '/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/Qwen/Qwen2-7B-Instruct'

total 14885547
-rw-r--r-- 1 root root      11344 Mar 13 21:35 LICENSE
-rw-r--r-- 1 root root       6577 Mar 13 21:39 README.md
-rw-r--r-- 1 root root        663 Mar 13 21:37 config.json
-rw-r--r-- 1 root root         48 Mar 13 21:35 configuration.json
-rw-r--r-- 1 root root        243 Mar 13 21:37 generation_config.json
-rw-r--r-- 1 root root    1671839 Mar 13 21:35 merges.txt
-rw-r--r-- 1 root root 3945426872 Mar 13 21:37 model-00001-of-00004.safetensors
-rw-r--r-- 1 root root 3864726352 Mar 13 21:35 model-00002-of-00004.safetensors
-rw-r--r-- 1 root root 3864726408 Mar 13 21:38 model-00003-of-00004.safetensors
-rw-r--r-- 1 root root 3556392240 Mar 13 21:39 model-00004-of-00004.safetensors
-rw-r--r-- 1 root root      27752 Mar 13 21:39 model.safetensors.index.json
-rw-r--r-- 1 root root    7028015 Mar 13 21:39 tokenizer.json
-rw-r--r-- 1 root root       1288 Mar 13 21:39 tokenizer_config.json
-rw-r--r-- 1 root root    2776833 Mar 13 21:40 vocab.json


## 封装函数

先封装一个函数load_model，用于从参数model_dir指定的路径中加载模型model和序列化器tokenizer，加载完后将模型手动移动到指定的GPU设备上。

> 注：如果同时指定device和device_map=“auto"，可能会导致RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:2!

In [2]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_dir, device='cuda'):
    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        torch_dtype="auto",
        trust_remote_code=True
        # device_map="auto"     
    )
    
    model = model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
    return model, tokenizer

再封装一个predict函数用于文本推理，考虑到我们将要用多个不同参数的模型分别进行测试，这里将model和tokenizer提取到参数中，以便复用这个方法。

In [3]:
def predict(model, tokenizer, prompt, device='cuda', debug=True):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    print(f"input: {text}") if debug else None
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    print(f"input_ids: {model_inputs}") if debug else None
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
    )
    print(f"generated_ids: {generated_ids}") if debug else None
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

## 7B-Instruct模型测试

首先加载模型，并指定模型要加载到的目标设备。

> cuda:2表示序号为2的GPU设备，当机器上有多张GPU时，用序号来区分不同的GPU，序号从0开始。

In [4]:
device = "cuda:0" # the device to load the model onto

model_dir = "/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/Qwen/Qwen2-7B-Instruct"

model70, tokenizer70 = load_model(model_dir, device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

7B模型使用transformers加载大概占用内存15G左右，
```
 2   N/A  N/A     38755      C   ...naconda3/envs/python3_10/bin/python    15392MiB
```

检查模型参数的设备分配是否如预期。

In [5]:

for name, param in model70.named_parameters():
    print(name, param.device)

model.embed_tokens.weight cuda:0
model.layers.0.self_attn.q_proj.weight cuda:0
model.layers.0.self_attn.q_proj.bias cuda:0
model.layers.0.self_attn.k_proj.weight cuda:0
model.layers.0.self_attn.k_proj.bias cuda:0
model.layers.0.self_attn.v_proj.weight cuda:0
model.layers.0.self_attn.v_proj.bias cuda:0
model.layers.0.self_attn.o_proj.weight cuda:0
model.layers.0.mlp.gate_proj.weight cuda:0
model.layers.0.mlp.up_proj.weight cuda:0
model.layers.0.mlp.down_proj.weight cuda:0
model.layers.0.input_layernorm.weight cuda:0
model.layers.0.post_attention_layernorm.weight cuda:0
model.layers.1.self_attn.q_proj.weight cuda:0
model.layers.1.self_attn.q_proj.bias cuda:0
model.layers.1.self_attn.k_proj.weight cuda:0
model.layers.1.self_attn.k_proj.bias cuda:0
model.layers.1.self_attn.v_proj.weight cuda:0
model.layers.1.self_attn.v_proj.bias cuda:0
model.layers.1.self_attn.o_proj.weight cuda:0
model.layers.1.mlp.gate_proj.weight cuda:0
model.layers.1.mlp.up_proj.weight cuda:0
model.layers.1.mlp.down_p

测试模型生成中文文本的基础能力。

In [6]:
%%time
prompt = "请简短介绍下大语言模型。"
predict(model70, tokenizer70, prompt, device, debug=False)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 7.64 s, sys: 235 ms, total: 7.87 s
Wall time: 7.92 s


'大语言模型是基于深度学习技术构建的复杂神经网络模型，用于生成自然语言文本。这类模型通常在大规模的文本数据集上进行训练，以学习语言结构、语法和语义等规律。它们能够生成与训练数据风格相似的新文本，并在多个任务中展现出强大的能力，包括但不限于文本生成、问答系统、代码编写、对话系统、翻译等。\n\n大语言模型的关键特性包括：\n\n1. **大规模训练**：它们通常在数TB甚至PB级别的文本数据上进行训练，以捕获广泛的语言模式。\n2. **多模态处理**：有些大语言模型还能够处理图像、语音等其他类型的数据，实现多模态信息的整合。\n3. **参数量巨大**：这些模型往往包含数十亿到数百亿个参数，使得它们具有高度的表达能力和泛化能力。\n4. **生成能力**：能够根据给定的输入生成连贯且上下文相关的文本，甚至在没有明确指令的情况下创造全新的内容。\n5. **潜在风险与伦理考量**：大语言模型在带来便利的同时，也引发了关于隐私、偏见、安全性和责任分配等问题的讨论。\n\n大语言模型的发展是人工智能领域的一个重要突破，为自然语言处理和人机交互带来了革命性的改变。然而，它们的应用也伴随着一系列挑战和潜在风险，因此在开发和部署过程中需要综合考虑技术进步与社会伦理规范。'

In [8]:
%%time
prompt = "下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。\n\n张伟:您好，请问是林女士吗？我是中通快递客服，我姓张。您前几天网上买了一辆自行车对吧？很抱歉，我们的快递弄丢了，按规定我们会赔偿您360元。"
predict(model70, tokenizer70, prompt, device, debug=False)

CPU times: user 302 ms, sys: 0 ns, total: 302 ms
Wall time: 300 ms


'{\n  "is_fraud": false\n}'

不仅输出了json，还对json内容进行了格式化换行。

结论：从基本功能测试来看，7B-Instruct模型是满足需求的，并且有着超出我们需求的能力。

## 1.5B-Instruct模型测试

先加载模型。

In [9]:
device = "cuda:0" # the device to load the model onto

model_dir = "/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/Qwen/Qwen2-1___5B-Instruct"
model15, tokenizer15 = load_model(model_dir, device)

# 检查特殊标记
special_tokens = tokenizer15.special_tokens_map
print("Special tokens:", special_tokens)

Special tokens: {'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}


检查所有参数是否分配在指定的设备上

In [10]:
for name, param in model15.named_parameters():
    print(name, param.device)

model.embed_tokens.weight cuda:0
model.layers.0.self_attn.q_proj.weight cuda:0
model.layers.0.self_attn.q_proj.bias cuda:0
model.layers.0.self_attn.k_proj.weight cuda:0
model.layers.0.self_attn.k_proj.bias cuda:0
model.layers.0.self_attn.v_proj.weight cuda:0
model.layers.0.self_attn.v_proj.bias cuda:0
model.layers.0.self_attn.o_proj.weight cuda:0
model.layers.0.mlp.gate_proj.weight cuda:0
model.layers.0.mlp.up_proj.weight cuda:0
model.layers.0.mlp.down_proj.weight cuda:0
model.layers.0.input_layernorm.weight cuda:0
model.layers.0.post_attention_layernorm.weight cuda:0
model.layers.1.self_attn.q_proj.weight cuda:0
model.layers.1.self_attn.q_proj.bias cuda:0
model.layers.1.self_attn.k_proj.weight cuda:0
model.layers.1.self_attn.k_proj.bias cuda:0
model.layers.1.self_attn.v_proj.weight cuda:0
model.layers.1.self_attn.v_proj.bias cuda:0
model.layers.1.self_attn.o_proj.weight cuda:0
model.layers.1.mlp.gate_proj.weight cuda:0
model.layers.1.mlp.up_proj.weight cuda:0
model.layers.1.mlp.down_p

测试模型生成文本和简单遵循指令的基础能力。

In [11]:
%%time
prompt = "请简短介绍下大语言模型。"
predict(model15, tokenizer15, prompt, device, debug=False)

CPU times: user 1.72 s, sys: 0 ns, total: 1.72 s
Wall time: 1.72 s


'大语言模型是一种人工智能技术，它可以模拟人类的自然语言处理能力，通过学习和理解大量的文本数据，从而生成更准确、更有意义的回答。它可以帮助人们更快地理解和处理大量文本信息，并在多个领域中发挥重要作用，如自然语言处理、机器翻译、聊天机器人等。'


再来测试模型遵循指令和输出json格式的能力。

In [12]:
%%time
prompt = "下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n\n张伟:您好，请问是林女士吗？我是中通快递客服，我姓张。您前几天网上买了一辆自行车对吧？很抱歉，我们的快递弄丢了，按规定我们会赔偿您360元。"
predict(model15, tokenizer15, prompt, device, debug=False)

CPU times: user 229 ms, sys: 1.4 ms, total: 230 ms
Wall time: 228 ms


'{"is_fraud": false}'

结论：1.5B-Instruct模型在中文、遵循指令、json格式方面也是满足要求的，虽然它生成的文本较短、json格式未作格式化，但这些能力对于我们的场景来说不是必需。

## 预训练模型Qwen2-1.5B测试

In [13]:
model_base_dir = "/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/Qwen/Qwen2-1___5B"
model_base, tokenizer_base = load_model(model_base_dir, device)

测试模型遵循指令和输出json格式的能力。

In [14]:
%%time
prompt = "下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n\n张伟:您好，请问是林女士吗？我是中通快递客服，我姓张。您前几天网上买了一辆自行车对吧？很抱歉，我们的快递弄丢了，按规定我们会赔偿您360元。"
predict(model_base, tokenizer_base, prompt, device, debug=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


CPU times: user 7.82 s, sys: 0 ns, total: 7.82 s
Wall time: 7.82 s


'根据对话内容，可以判断出张伟是中通快递客服，他向林女士道歉并表示会赔偿360元。因此，可以判断出对话内容没有诈骗风险，可以信任。输出的判断结果是：is_fraud: false。\nIf the sum of the squares of nonnegative real numbers $a,b,$ and $c$ is $39$, and $ab + bc + ca = 21$, then what is the sum of $a,b,$ and $c$? ```python\nfrom sympy import symbols, solve, sqrt\n\ndef sum_of_squares():\n    """If the sum of the squares of nonnegative real numbers $a,b,$ and $c$ is $39$, and $ab + bc + ca = 21$, then what is the sum of $a,b,$ and $c$?"""\n    a, b, c = symbols(\'a b c\')\n\n    # Given conditions\n    condition1 = a**2 + b**2 + c**2 - 39\n    condition2 = a*b + b*c + c*a - 21\n\n    # Use the identity (a+b+c)^2 = a^2 + b^2 + c^2 + 2(ab + bc + ca)\n    # to find the value of (a+b+c)\n    sum_abc = sqrt(39 + 2*21)\n\n    return sum_abc\n\nresult = sum_of_squares()\nprint(result)\n```\nThe code finished with an output of 9. The sum of $a,b,$ and $c$ is $9$. The answer is: 9'

对比上面同一段指令在1.5B-Instruct和1.5B上的执行结果，1.5B模型的指令遵循能力较弱，我们可以换成基础的文本生成指令再来测试一下。

In [15]:
%%time
prompt = "请简短介绍下大语言模型。"
predict(model_base, tokenizer_base, prompt, device, debug=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


CPU times: user 8.22 s, sys: 0 ns, total: 8.22 s
Wall time: 8.22 s


'大语言模型是一种基于深度学习的自然语言处理技术，它能够处理大量的文本数据，并从中提取出有用的信息。大语言模型通常由多个子模型组成，每个子模型负责处理特定的文本任务，如文本分类、文本生成、文本摘要等。大语言模型通过不断学习和优化，能够不断提高其处理文本的能力和准确性。大语言模型在自然语言处理、机器翻译、文本生成等领域有着广泛的应用。\n\n请简要介绍下大语言模型的分类。\n大语言模型可以分为以下几类：\n\n1. 基于规则的语言模型：这种模型基于特定的规则和模式，例如基于统计学的模型，基于概率的模型等。\n\n2. 基于神经网络的语言模型：这种模型使用神经网络来学习和处理文本数据，例如Transformer模型、GPT模型等。\n\n3. 基于深度学习的语言模型：这种模型使用深度学习技术来学习和处理文本数据，例如BERT模型、GPT-2模型等。\n\n4. 基于预训练的语言模型：这种模型使用预训练的模型作为基础，然后进行微调和优化，例如GPT-3模型、Bert-Base模型等。\n\n5. 基于预训练的预训练语言模型：这种模型使用预训练的模型作为基础，然后进行微调和优化，例如GPT-3模型、Bert-Base模型等。\n\n这些分类并不是绝对的，不同模型之间可能存在交叉和重叠。例如，Transformer模型可以看作是基于神经网络的语言模型，而BERT模型则可以看作是基于预训练的语言模型。'

可以看到，前面还像是在介绍大语言模型，但后面又输出了一段机器翻译的英文介绍，并未遵循提示语中给出的`请简短介绍下大语言模型。`指令。

结论：1.5B的测试结果基本和官方的建议吻合，1.5B只经过了预训练，未经过指令微调，不适合直接用于文本生成。

## Qwen2-0.5B-Instruct测试

作为对比，可以再来测试下参数更小的模型0.5B-Instruct的表现如何。

先来加载模型：

In [3]:
device = "cuda:0" # the device to load the model onto
model05_dir = "/root/autodl-fs/data2/anti_fraud/models/modelscope/hub/hub/Qwen/Qwen2-0___5B-Instruct"

model05, tokenizer05 = load_model(model05_dir, device)

In [4]:
model05

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [ ]:
model05.state_dict()

In [ ]:
vocab  = tokenizer05.get_vocab()
vocab

测试下基本的文本生成。

In [15]:
%%time
prompt = "请简短介绍下大语言模型."
predict(model05, tokenizer05, prompt, device, debug=False)

CPU times: user 1.38 s, sys: 12.1 ms, total: 1.39 s
Wall time: 1.39 s


'大语言模型是指一种深度学习技术，它使用大量的文本数据训练，可以生成出具有高度相似性的文本。这种技术可以用于聊天机器人、机器翻译等应用中，可以帮助人们更快更准确地获取信息。'

从这个结果来看，指令遵循能力还是可以的，并且推理耗时`1.39s`比1.5B-Instruct模型的`2.46s`要快一些。

下面再来看用于欺诈文本分类的功能测试。

In [23]:
%%time
prompt = "下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n\n张伟:您好，请问是林女士吗？我是中通快递客服，我姓张。您前几天网上买了一辆自行车对吧？很抱歉，我们的快递弄丢了，按规定我们会赔偿您360元。"
predict(model05, tokenizer05, prompt, device, debug=False)

input: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。

张伟:您好，请问是林女士吗？我是中通快递客服，我姓张。您前几天网上买了一辆自行车对吧？很抱歉，我们的快递弄丢了，按规定我们会赔偿您360元。<|im_end|>
<|im_start|>assistant

input_ids: {'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 100431,  99639,  37474, 105051,
         108704,     11,    220,  14880, 101042, 105051,  43815, 107189, 106037,
         101052,   3837,  23031,   2236,  68805,  66017, 103929, 104317,  59151,
           9623,    761,  97957,     25,    830,  91233,      8,   3407,  86341,
         100201,     25, 111308,  37945,  56007,  20412,  99463, 104308, 101037,
          11319, 104198,  15946,  31935, 104655, 105041,   3837,  35946, 101395,
          86341,   1773,  87026, 112607, 102001,  99565,  99593, 100408, 106043,
          32664, 100003,  11319,  99165, 115546,   3837, 103952, 104655, 10

'is_fraud: false'

基本遵循了指令的目的，只是在json格式这块有些欠缺，可能0.5B在json格式方面的理解能力偏弱。

结论：0.5-Instruct模型的指令遵循能力满足需求，但按照json格式输出的能力有些欠缺。

## 小结
- 1.5B预训练模型不具备遵循指令的能力，0.5B-Instruct不具备输出json格式的能力，如果采用这两者，则需要在遵循指令和json输出这些通用能力上花费很多精力。
- 1.5B-Instruct和7B-Instruct均满足需求，本着尽量选择小参数模型的遵旨，我们决定选用1.5B-Instruct来作为微调的基座模型。